In [19]:

from dotenv import load_dotenv
import os
import geopandas as gpd
import censusdis.data as ced
from censusdis import states
from data.constants import LOCAL_CRS

load_dotenv()


True

In [22]:
tracts = gpd.read_file("../../data/raw/tracts.geojson").to_crs(LOCAL_CRS)

In [27]:
DATASET = "acs/acs5"
YEAR = 2020

TOTAL_POPULATION_VARIABLE = "B01003_001E"
MEDIAN_HOUSEHOLD_INCOME_VARIABLE = "B19013_001E"
VARIABLES = ["NAME", TOTAL_POPULATION_VARIABLE]

pop = ced.download(
    DATASET,
    YEAR,
    VARIABLES,
    state=states.IL,
    county="031",
    tract="*",
    with_geometry=True
)
pop = (pop.assign(centroid = lambda x: x['geometry'].to_crs(LOCAL_CRS).centroid)
       .set_geometry('centroid')
       .drop(columns=['geometry'])
       .rename_geometry('geometry')
       .rename(columns={'B01003_001E':'total_pop'}))


In [28]:
tract_pop = tracts.sjoin(pop[['geometry','total_pop']], how='left', predicate='contains')